<a href="https://colab.research.google.com/github/ZahibVentura/Simulaci-n-1/blob/main/Inventario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import random as rd
import matplotlib.pyplot as plt    #Importamos las librerias a utilizar
import numpy as np
from prettytable import PrettyTable as pt

###Metodología

Para este problema se debe simular la distribución del ejercicio, tanto para la dsitribución de probabilidad empírica de la demanda como la de los meses.

Hay que considerar los factores estacionarios de cada uno de los meses de cada año.

Simulamos la tabla para un solo año y calculamos costos de ese año.

Se simula para $N$ años y ver cual es el costo total de la población.

Aplicamos el algoritmo de Hooke-Jeeves para hallar la solución optima

In [87]:
cantidad = [35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60] #lista de cantidades
probabilidad = [0.010,0.015,0.020,0.020,0.022,0.023,0.025,0.027,0.028,0.029,0.035,0.045,             #probabilidad de cada cantidad
                0.060,0.065,0.070,0.080,0.075,0.070,0.065,0.06,0.050,0.040,0.030,0.016,0.015,0.005]

In [88]:
#Guarda la acumulada de la lista de probabilidad
L_acumulada = []
acum = 0
for i in probabilidad:
    acum += i
    L_acumulada.append(acum)

In [89]:
#Funcion que devuelve un valor de la ista cantidad dependiendo la lista acumulada
def acumulada_inversa(u1=0.5):  #se define la función
  for i in range(len(cantidad)):
    if u1 < L_acumulada[i]:
      return cantidad[0]
    elif L_acumulada[i] <= u1 < L_acumulada[i+1]:
      return cantidad[i+1]

In [90]:

#Tiempos de entrega

tiempo_entrega = [1,2,3]
proba_entrega = [0.3,0.4,0.3]

L_proba_entrega = []
acum = 0
for i in proba_entrega:
    acum += i
    L_proba_entrega.append(acum)

In [91]:
#Funcion que nos devuleve un tiempo de entrega respecto a su probabilidad
def proba_inversa(u2 = 0.5):
  for i in range(len(tiempo_entrega)):
    if u2 < L_proba_entrega[i]:
      return tiempo_entrega[0]
    elif  L_proba_entrega[i] <= u2 < L_proba_entrega[i + 1]:
      return tiempo_entrega[i + 1]

In [96]:
# Lista y variables del ejercicio
factor_estacionario = [1.2, 1.0, 0.9, 0.8, 0.8, 0.7, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4]
No_de_unidades_a_ordenar = 200
inventario_inicial = 150
reorden = 100

In [97]:
# Variables de control
pedido_pendiente = False
tiempo_pedido_pendiente = 0
Total_de_ordenes = 0
Falta_total = 0
Total_IMP = 0

In [98]:
# Inicialización de la tabla
Tabla = pt(['Mes', 'Inv.inicial', 'No.aleatorio', 'Demanda', 'Inv.final', 'Faltante', 'Orden', 'Inv.mensual promedio'])

# Simulación del inventario, solo devuelve una tabla por cada vez que se ejecute el programa
for i in range(12):
    aux = inventario_inicial
    No_aleatorio = rd.uniform(0, 1)
    Demanda_ajustada = int(acumulada_inversa(No_aleatorio) * factor_estacionario[i])
    inventario_final = inventario_inicial - Demanda_ajustada

    if inventario_final > 0:
      inventario_mensual_promedio = (aux + inventario_final) / 2
      Total_IMP += inventario_mensual_promedio
    else:
      inventario_mensual_promedio = round((aux / 2) * (aux / Demanda_ajustada))
      Total_IMP += inventario_mensual_promedio

    if inventario_final >= 0:
        faltante = 0
    else:
        faltante = abs(inventario_final)
        Falta_total += faltante
        inventario_final = 0



    # Control del pedido pendiente
    if pedido_pendiente:
        tiempo_pedido_pendiente -= 1
        if tiempo_pedido_pendiente == 0:
            inventario_final += No_de_unidades_a_ordenar
            pedido_pendiente = False

    # Revisión de la necesidad de reordenar
    if inventario_final < reorden and not pedido_pendiente:
        aleator = rd.uniform(0, 1)
        tiempo_pedido_pendiente = proba_inversa(aleator)
        pedido_pendiente = True
        orden = 1
        Total_de_ordenes += 1
    else:
        orden = 0



    # Impresión de la tabla
    lista_tabla = [i + 1, aux, No_aleatorio, Demanda_ajustada, inventario_final, faltante, orden, inventario_mensual_promedio]
    Tabla.add_row(lista_tabla)

    inventario_inicial = inventario_final

#Imprimimos la tabla final
print(Tabla)
print("Total de ordenes:", Total_de_ordenes)
print("Falta total:", Falta_total)
print("Total de Imp:", Total_IMP)

#Se calcula cada costo y así como el costo total del inventario
Costo_llevar_inventario = Total_IMP*1.67
Costo_faltante = Falta_total*50
Costo_ordenar = Total_de_ordenes*100
Costo_total = Costo_llevar_inventario + Costo_faltante + Costo_ordenar
print(' Costo total de ordenes: ',Costo_ordenar)
print(' Costo total de llevar inventario: ',Costo_llevar_inventario)
print(' Costo total de faltante: ',Costo_faltante)
print(' Costo total: ',Costo_total)

+-----+-------------+---------------------+---------+-----------+----------+-------+----------------------+
| Mes | Inv.inicial |     No.aleatorio    | Demanda | Inv.final | Faltante | Orden | Inv.mensual promedio |
+-----+-------------+---------------------+---------+-----------+----------+-------+----------------------+
|  1  |     150     |  0.4273271072055963 |    58   |     92    |    0     |   1   |        121.0         |
|  2  |      92     |  0.5252006916313261 |    50   |    242    |    0     |   0   |         67.0         |
|  3  |     242     | 0.29715843935146236 |    41   |    201    |    0     |   0   |        221.5         |
|  4  |     201     |  0.0898208648972163 |    32   |    169    |    0     |   0   |        185.0         |
|  5  |     169     |  0.8197200175306696 |    43   |    126    |    0     |   0   |        147.5         |
|  6  |     126     | 0.15829798003710605 |    29   |     97    |    0     |   1   |        111.5         |
|  7  |      97     |  0.543

In [105]:
#Simulando el problema varias veces para ver cual puede ser el costo total final asi como los
# respectivos costos

#La razon del porque hacerlo es para podes estimas un valor que calcule la media de cada costo
def simulacion(No_de_unidades_a_ordenar, reorden, n=50): #se define una función cuyo parametros son las unidades a ordenar, el reorden y el numero de iteraciones
    LISTA1 = []
    LISTA2 = []
    LISTA3 = []
    LISTA4 = [] #se definen listas y variables a ocupar
    SUM1 = 0
    SUM2 = 0
    SUM3 = 0
    SUM4 = 0
    t = 1
    for i in range(n):
        factor_estacionario = [1.2, 1.0, 0.9, 0.8, 0.8, 0.7, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4]
        No_de_unidades_a_ordenar = 200
        inventario_inicial = 150
        reorden = 100
        pedido_pendiente = False         #son los valores de la tabla, se mandan a llamar nuevamente
        tiempo_pedido_pendiente = 0
        Total_de_ordenes = 0
        Falta_total = 0
        Total_IMP = 0
        for j in range(12):
            aux = inventario_inicial
            No_aleatorio = rd.uniform(0, 1)
            Demanda_ajustada = int(acumulada_inversa(No_aleatorio) * factor_estacionario[j])
            inventario_final = inventario_inicial - Demanda_ajustada

            if inventario_final > 0:
              inventario_mensual_promedio = (aux + inventario_final) / 2
              Total_IMP += inventario_mensual_promedio
            else:
              inventario_mensual_promedio = round((aux / 2) * (aux / Demanda_ajustada))
              Total_IMP += inventario_mensual_promedio

            if inventario_final >= 0:
                faltante = 0
            else:
                faltante = abs(inventario_final)
                Falta_total += faltante
                inventario_final = 0



            # Control del pedido pendiente
            if pedido_pendiente:
                tiempo_pedido_pendiente -= 1
                if tiempo_pedido_pendiente == 0:
                    inventario_final += No_de_unidades_a_ordenar
                    pedido_pendiente = False

            # Revisión de la necesidad de reordenar
            if inventario_final < reorden and not pedido_pendiente:
                aleator = rd.uniform(0, 1)
                tiempo_pedido_pendiente = proba_inversa(aleator)
                pedido_pendiente = True
                orden = 1
                Total_de_ordenes += 1
                #print(tiempo_pedido_pendiente)
            else:
                orden = 0


            inventario_inicial = inventario_final

        Costo_llevar_inventario = Total_IMP * 1.67
        Costo_faltante = Falta_total * 50
        Costo_ordenar = Total_de_ordenes * 100
        Costo_total = Costo_llevar_inventario + Costo_faltante + Costo_ordenar

        SUM1 += Costo_ordenar
        SUM2 += Costo_llevar_inventario
        SUM3 += Costo_faltante
        SUM4 += Costo_total


        LISTA1.append(SUM1 / (t))
        LISTA2.append(SUM2 / (t))
        LISTA3.append(SUM3 / (t))
        LISTA4.append(SUM4 / (t))
        t=t+1

    return LISTA1[-1], LISTA2[-1], LISTA3[-1], LISTA4[-1]



In [109]:
# Ejecutar simulación
LIS1, LIS2, LIS3, LIS4 = simulacion(200,100,5000)
#imprimir valores
print("Costo de ordenar promedio:", LIS1)
print("Costo de llevar inventario promedio:", LIS2)
print("Costo faltante promedio:", LIS3)
print("Costo total promedio:", LIS4)

Costo de ordenar promedio: 297.68
Costo de llevar inventario promedio: 1987.9997299999964
Costo faltante promedio: 2422.15
Costo total promedio: 4707.829729999995


In [119]:
# Algoritmo de Hooke-Jeeves
def hooke_jeeves(f, x0, delta=0.5, epsilon=1e-5, max_iter=100):
    n = len(x0)
    x = np.array(x0)
    xp = np.array(x0)
    xb = np.array(x0)
    for i in range(max_iter):
        for j in range(n):
            d = np.zeros(n)
            d[j] = delta
            if f(xb + d) < f(xb):
                xb = xb + d
            elif f(xb - d) < f(xb):
                xb = xb - d
        if np.linalg.norm(xb - xp) < epsilon:
            break
        xp = xb + (xb - xp)
        if f(xp) < f(xb):
            xb = xp
        x = xb
    return xb

# Función de costo total para optimizar
def costo_total(params):
    No_de_unidades_a_ordenar, reorden = params
    _, _, _, Costo_total = simulacion(No_de_unidades_a_ordenar, reorden,1000)
    return Costo_total

# Parâmetros iniciales: [No_de_unidades_a_ordenar, reorden]
x0 = [200, 100]

# Encontrar el óptimo
optimo = hooke_jeeves(costo_total, x0)
No_de_unidades_a_ordenar_optimo, reorden_optimo = optimo

print(f"Óptimo No_de_unidades_a_ordenar: {No_de_unidades_a_ordenar_optimo}")
print(f"Óptimo reorden: {reorden_optimo}")

# Calcular los costos con los valores óptimos
Costo_ordenar_optimo, Costo_llevar_inventario_optimo, Costo_faltante_optimo, Costo_total_optimo = simulacion(No_de_unidades_a_ordenar_optimo, reorden_optimo)
print(f"Costo a ordenar óptimo: {Costo_ordenar_optimo}")
print(f"Costo de llevar el inventario óptimo: {Costo_llevar_inventario_optimo}")
print(f"Costo faltante óptimo: {Costo_faltante_optimo}")
print(f"Costo total óptimo: {Costo_total_optimo}")

Óptimo No_de_unidades_a_ordenar: 201.0
Óptimo reorden: 100.0
Costo a ordenar óptimo: 298.0
Costo de llevar el inventario óptimo: 2041.6752000000001
Costo faltante óptimo: 2317.0
Costo total óptimo: 4656.675199999999
